In [46]:
import tensorflow as tf
import numpy as np
import math
import os
import random
import numpy as np
import pandas as pd
from pandas import DataFrame
import  pickle
import math
import copy

In [47]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [48]:
with open('../data/state_features.txt') as f:
    state_features = f.read().split()
print (state_features)

['Albumin', 'Arterial_BE', 'Arterial_lactate', 'Arterial_pH', 'BUN', 'CO2_mEqL', 'Calcium', 'Chloride', 'Creatinine', 'DiaBP', 'FiO2_1', 'GCS', 'Glucose', 'HCO3', 'HR', 'Hb', 'INR', 'Ionised_Ca', 'Magnesium', 'MeanBP', 'PT', 'PTT', 'PaO2_FiO2', 'Platelets_count', 'Potassium', 'RR', 'SGOT', 'SGPT', 'SIRS', 'SOFA', 'Shock_Index', 'Sodium', 'SpO2', 'SysBP', 'Temp_C', 'Total_bili', 'WBC_count', 'Weight_kg', 'age', 'elixhauser', 'gender', 'mechvent', 'output_4hourly', 'output_total', 'paCO2', 'paO2', 're_admission', 'bloc']


In [49]:
df = pd.read_csv('../data/rl_train_data_final_cont.csv')

In [50]:
df.head()

,bloc,icustayid,charttime,gender,age,elixhauser,re_admission,died_in_hosp,died_within_48h_of_out_time,mortality_90d,...,input_total,input_4hourly,output_total,output_4hourly,cumulated_balance,SOFA,SIRS,vaso_input,iv_input,reward
0,0.000000,11,6898241400,1.0,0.902327,0.428571,1.0,0,0,0,...,0.0,0.0,0.000000,0.000000,0.223817,0.500000,0.00,0.0,0.0,0.383136
1,0.222560,11,6898255800,1.0,0.902327,0.428571,1.0,0,0,0,...,0.0,0.0,0.574861,0.707254,0.223281,0.416667,0.00,0.0,0.0,-0.546041
2,0.356608,11,6898270200,1.0,0.902327,0.428571,1.0,0,0,0,...,0.0,0.0,0.629131,0.723746,0.222629,0.416667,0.25,0.0,0.0,-0.884898
3,0.452837,11,6898284600,1.0,0.902327,0.428571,1.0,0,0,0,...,0.0,0.0,0.659814,0.726688,0.221953,0.458333,0.25,0.0,0.0,-0.025000
4,0.527957,11,6898299000,1.0,0.902327,0.428571,1.0,0,0,0,...,0.0,0.0,0.675701,0.699627,0.221464,0.458333,0.25,0.0,0.0,0.125000


In [51]:
val_df = pd.read_csv('../data/rl_val_data_final_cont.csv')

In [52]:
test_df = pd.read_csv('../data/rl_test_data_final_cont.csv')

In [53]:
# define an action mapping - how to get an id representing the action from the (iv,vaso) tuple
action_map = {}
count = 0
for iv in range(5):
    for vaso in range(5):
        action_map[(iv,vaso)] = count
        count += 1

In [54]:
inv_action_map = {}
for iv in range(5):
    for vaso in range(5):
        inv_action_map[5*iv+vaso] = [iv,vaso]    

In [55]:
# load in the actions from the q network
q_net_save_dir = '../continuous/dqn_normalT/'
train_actions = pickle.load(open( q_net_save_dir + "dqn_normal_actions_train.p", "rb" ))
val_actions = pickle.load(open( q_net_save_dir + "dqn_normal_actions_val.p", "rb" ))
test_actions = pickle.load(open( q_net_save_dir + "dqn_normal_actions_test.p", "rb" ))

df['agent_actions'] = train_actions
df['agent_iv'] = df['agent_actions'].apply(lambda x:inv_action_map[x][0] )
df['agent_vaso'] = df['agent_actions'].apply(lambda x:inv_action_map[x][1] )

val_df['agent_actions'] = val_actions
val_df['agent_iv'] = val_df['agent_actions'].apply(lambda x:inv_action_map[x][0] )
val_df['agent_vaso'] = val_df['agent_actions'].apply(lambda x:inv_action_map[x][1] )

test_df['agent_actions'] = test_actions
test_df['agent_iv'] = test_df['agent_actions'].apply(lambda x:inv_action_map[x][0] )
test_df['agent_vaso'] = test_df['agent_actions'].apply(lambda x:inv_action_map[x][1] )

In [56]:
# preprocess the actions so that they are zero mean
df['iv_input'] = df['iv_input'].apply(lambda x: x/4.0)
df['vaso_input'] = df['vaso_input'].apply(lambda x: x/4.0)

val_df['iv_input'] = val_df['iv_input'].apply(lambda x: x/4.0)
val_df['vaso_input'] = val_df['vaso_input'].apply(lambda x: x/4.0)

test_df['iv_input'] = test_df['iv_input'].apply(lambda x: x/4.0)
test_df['vaso_input'] = test_df['vaso_input'].apply(lambda x: x/4.0)

df['agent_iv'] = df['agent_iv'].apply(lambda x: x/4.0)
df['agent_vaso'] = df['agent_vaso'].apply(lambda x: x/4.0)

val_df['agent_iv'] = val_df['agent_iv'].apply(lambda x: x/4.0)
val_df['agent_vaso'] = val_df['agent_vaso'].apply(lambda x: x/4.0)

test_df['agent_iv'] = test_df['agent_iv'].apply(lambda x: x/4.0)
test_df['agent_vaso'] = test_df['agent_vaso'].apply(lambda x: x/4.0)

In [57]:
# Add Gaussian noise to the state features
gaussian_shape = df.loc[:, state_features].values.shape
noise = np.random.normal(0, 0.03, gaussian_shape)
df.loc[:, state_features] += noise

In [58]:
hidden_1_size = 500
hidden_2_size = 500
class EnvModel():
    def __init__(self):
        self.phase = tf.placeholder(tf.bool)
        
        self.input_size = len(state_features)

        self.cur_state = tf.placeholder(tf.float32, shape=[None, self.input_size],name="cur_state")
        self.next_state = tf.placeholder(tf.float32, shape=[None, self.input_size],name="next_state")
        
        self.done_flags = tf.placeholder(tf.int32, shape=[None], name="done_flags")
        
        self.actions = tf.placeholder(tf.int32, shape = [None, 2], name="actions")
        
        self.input = tf.concat([self.cur_state, tf.cast(self.actions, tf.float32)], axis=1)
        
        self.target = self.next_state - self.cur_state

        self.fc_1 = tf.contrib.layers.fully_connected(self.input, hidden_1_size, activation_fn=tf.nn.relu)
        self.fc_1_bn = tf.contrib.layers.batch_norm(self.fc_1, center=True, scale=True, is_training=self.phase)
        self.fc_2 = tf.contrib.layers.fully_connected(self.fc_1_bn, hidden_2_size, activation_fn=tf.nn.relu)
        self.fc_2_bn = tf.contrib.layers.batch_norm(self.fc_2, center=True, scale=True, is_training=self.phase)
        
        self.output = tf.contrib.layers.fully_connected(self.fc_2_bn, self.input_size, activation_fn = None)
        
        self.multiplier = tf.expand_dims(1 -self.done_flags, 1)
        
        self.loss = tf.reduce_mean(tf.multiply(tf.square(self.target-self.output), tf.cast(self.multiplier, tf.float32)))  
        
        self.est_next_state = self.output + self.cur_state  #下一个状态 = 预测增量 ＋ 当前状态

        self.trainer = tf.train.AdamOptimizer(learning_rate=0.001)
        self.update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(self.update_ops):
        # Ensures that we execute the update_ops before performing the model update, so batchnorm works
            self.update_model = self.trainer.minimize(self.loss)

In [59]:
def process_train_batch(size):
    a = df.sample(n=size)
    states = None
    actions = None
    rewards = None
    next_states = None
    done_flags = None
    for i in a.index:
        cur_state = a.loc[i,state_features]
        iv = int(a.loc[i, 'iv_input'])
        vaso = int(a.loc[i, 'vaso_input'])
        action = np.array([iv,vaso])

        if i != df.index[-1]:
            # if not terminal step in trajectory             
            if df.loc[i, 'icustayid'] == df.loc[i+1, 'icustayid']:
                next_state = df.loc[i + 1, state_features]
                done = 0
            else:
                # trajectory is finished
                next_state = np.zeros(len(cur_state))
                done = 1
        else:
            # last entry in df is the final state of that trajectory
            next_state = np.zeros(len(cur_state))
            done = 1

        if states is None:
            states = copy.deepcopy(cur_state)
        else:
            states = np.vstack((states,cur_state))

        if actions is None:
            actions = [action]
        else:
            actions = np.vstack((actions,action))

        if next_states is None:
            next_states = copy.deepcopy(next_state)
        else:
            next_states = np.vstack((next_states,next_state))

        if done_flags is None:
            done_flags = [done]
        else:
            done_flags = np.vstack((done_flags,done))
    
    return (states, np.squeeze(actions), next_states, np.squeeze(done_flags), a)

In [60]:
# extract chunks of length size from the relevant dataframe, and yield these to the caller
# final tells us if we want to use AGENT actions, not physician actions
def process_eval_batch(size, eval_type = None, final=False):
    if eval_type is None:
        raise Exception('Provide eval_type to process_eval_batch')
    elif eval_type == 'train':
        a = df.copy()
    elif eval_type == 'val':
        a = val_df.copy()
    elif eval_type == 'test':
        a = test_df.copy()
    else:
        raise Exception('Unknown eval_type')
    count = 0
    while count < len(a.index):
        states = None
        actions = None
        rewards = None
        next_states = None
        done_flags = None

        start_idx = count
        end_idx = min(len(a.index), count+size)
        segment = a.index[start_idx:end_idx]
        
        for i in segment:
            cur_state = a.loc[i,state_features]
            
            if not final:
                iv = int(a.loc[i, 'iv_input'])
                vaso = int(a.loc[i, 'vaso_input'])
                action = np.array([iv,vaso])
            else:
                iv = int(a.loc[i, 'agent_iv'])
                vaso = int(a.loc[i, 'agent_vaso'])
                action = np.array([iv,vaso])
            reward = a.loc[i,'reward']

            if i != a.index[-1]:
                # if not terminal step in trajectory             
                if a.loc[i, 'icustayid'] == a.loc[i+1, 'icustayid']:
                    next_state = a.loc[i + 1, state_features]
                    done = 0
                else:
                    # trajectory is finished
                    next_state = np.zeros(len(cur_state))
                    done = 1
            else:
                # last entry in df is the final state of that trajectory
                next_state = np.zeros(len(cur_state))
                done = 1

            if states is None:
                states = copy.deepcopy(cur_state)
            else:
                states = np.vstack((states,cur_state))

            if actions is None:
                actions = [action]
            else:
                actions = np.vstack((actions,action))

            if next_states is None:
                next_states = copy.deepcopy(next_state)
            else:
                next_states = np.vstack((next_states,next_state))

            if done_flags is None:
                done_flags = [done]
            else:
                done_flags = np.vstack((done_flags,done))

        yield (states, np.squeeze(actions), next_states, np.squeeze(done_flags), a)
        
        count += size


In [61]:
def do_eval(eval_type, final=False):
    gen = process_eval_batch(size = 1000, eval_type=eval_type, final=final)

    error_ret = 0
    est_next_states = []

    for b in gen:

        states,actions,next_states, done_flags, _ = b

        est_next_state,loss = sess.run([env_model.est_next_state,env_model.loss], \
            feed_dict={env_model.cur_state:states,
                       env_model.next_state:next_states, 
                       env_model.actions:actions,
                       env_model.done_flags:done_flags,
                       env_model.phase:False})    
        error_ret += loss
        est_next_states.append(est_next_state)
  
    return np.array(est_next_states), error_ret

In [62]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # Don't use all GPUs 
config.allow_soft_placement = True  # Enable manual control

In [63]:
def do_save_results():
    # get the estimated next states UNDER THE AGENT POLICY based on the trained environment model
    est_next_states_train, _ = do_eval(eval_type = 'train', final=True)        
    est_next_states_val, _ = do_eval(eval_type = 'val', final=True)        
    est_next_states_test, _ = do_eval(eval_type = 'test', final=True)   
    
    # save everything for later - they're used in policy evaluation and when generating plots
    with open(save_dir + 'est_next_states_train.p', 'wb') as f:
        pickle.dump(est_next_states_train, f)
    with open(save_dir + 'est_next_states_val.p', 'wb') as f:
        pickle.dump(est_next_states_val, f)
    with open(save_dir + 'est_next_states_test.p', 'wb') as f:
        pickle.dump(est_next_states_test, f)

    return

In [64]:
# The main training loop is here
batch_size = 32
num_steps = 60000 # How many steps to train for
load_model = True #Whether to load a saved model.
save_dir = "./env_model_regression/"
save_path = "./env_model_regression/ckpt"#The path to save our model to.rk
tf.reset_default_graph()
env_model = EnvModel()
save_results = False

saver = tf.train.Saver(tf.global_variables())

init = tf.global_variables_initializer()

trainables = tf.trainable_variables()

#Make a path for our model to be saved in.
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

with tf.Session(config=config) as sess:
    if load_model == True:
        print('Trying to load model...')
        try:
            restorer = tf.train.import_meta_graph(save_path + '.meta')
            restorer.restore(sess, tf.train.latest_checkpoint(save_dir))
            print ("Model restored")
        except IOError:
            print ("No previous model found, running default init")
            sess.run(init)
    else:
        print("Running default init")
        sess.run(init)
    print("Init done")
    
    net_loss = 0.0
    for i in range(num_steps):
        if save_results:
            print ("Calling do save results")
            do_save_results()
            break
        
        states,actions,next_states, done_flags, sampled_df = process_train_batch(batch_size)

        # Train with the batch
        _,loss = sess.run([env_model.update_model,env_model.loss], \
            feed_dict={env_model.cur_state:states,
                       env_model.next_state:next_states, 
                       env_model.actions:actions,
                       env_model.done_flags:done_flags,
                       env_model.phase:True})

        net_loss += loss
    
        
        if i % 1000 == 0 and i > 0:
            saver.save(sess,save_path)
            print("Saved Model, step is " + str(i))
            
            av_loss = net_loss/(1000.0 * batch_size)
            print("Average loss is ", av_loss)
            net_loss = 0.0
        
        if (i % 10000==0) and i > 0:
            # run an evaluation on the validation set
            _, error = do_eval(eval_type = 'val')        
            print (error)
#             if (i % 30000==0) and i > 0:
#                 print ("Saving results")
#                 do_save_results()
    do_save_results()
    

Trying to load model...
INFO:tensorflow:Restoring parameters from ./env_model_regression/ckpt
Model restored
Init done
Saved Model, step is 1000
Average loss is  0.00021866526066878579
Saved Model, step is 2000
Average loss is  0.00021756604203255846
Saved Model, step is 3000
Average loss is  0.00021872609859565273
Saved Model, step is 4000
Average loss is  0.0002195398225230747
Saved Model, step is 5000
Average loss is  0.00021821961348177866
Saved Model, step is 6000
Average loss is  0.00021728554432775128
Saved Model, step is 7000
Average loss is  0.00021777011276572013
Saved Model, step is 8000
Average loss is  0.0002188581334630726
Saved Model, step is 9000
Average loss is  0.00021779590588266728
Saved Model, step is 10000
Average loss is  0.00021749177171295742


c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


0.1693765469826758
Saved Model, step is 11000
Average loss is  0.00021920736598258372
Saved Model, step is 12000
Average loss is  0.0002200252209586324
Saved Model, step is 13000
Average loss is  0.00022087058522447477
Saved Model, step is 14000
Average loss is  0.00021969152831297835
Saved Model, step is 15000
Average loss is  0.00021973035560222343
Saved Model, step is 16000
Average loss is  0.000218950066526304
Saved Model, step is 17000
Average loss is  0.0002172159703623038
Saved Model, step is 18000
Average loss is  0.00021875704808189765
Saved Model, step is 19000
Average loss is  0.00021846508974704193
Saved Model, step is 20000
Average loss is  0.00021955904927744996
0.17456511408090591
Saved Model, step is 21000
Average loss is  0.0002205288192781154
Saved Model, step is 22000
Average loss is  0.0002190678127662977
Saved Model, step is 23000
Average loss is  0.00021837765781674534
Saved Model, step is 24000
Average loss is  0.00021787460328050656
Saved Model, step is 25000
Av